In [2]:
import requests
import time
from lxml import etree
import tesserocr
from PIL import Image
from selenium import webdriver
from time import sleep
from lxml import etree
from selenium.webdriver.support.select import Select

def auto():
    chromeOptions = webdriver.ChromeOptions()
    prefs = {"download.default_directory": r"O:\土壤修复"}
    chromeOptions.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(chrome_options=chromeOptions)
    driver.get("https://kns.cnki.net/kns/brief/result.aspx?dbprefix=SCDB&crossDbcodes=CJFQ,CDFD,CMFD,CPFD,IPFD,CCND,CCJD")
    qikan = driver.find_elements_by_xpath('//*[@id="CJFQ"]/a')[0]  # 寻找
    qikan.click()
    sleep(1.5)
    opt = driver.find_element_by_id('txt_1_sel')
    Select(opt).select_by_visible_text('篇名')
    keywords1 = driver.find_element_by_id('txt_1_value1')  # 寻找第一个关键词输入框
    keywords1.send_keys("土壤修复")  # 传值
    check1 = driver.find_element_by_id("mediaBox3")
    check2 = driver.find_element_by_id("mediaBox4")
    check1.click()
    check2.click()
    btn1 = driver.find_element_by_id("btnSearch")  # 寻找检索按钮
    btn1.click()  # 点击检索
    sleep(1.5)

    driver.switch_to.frame('iframeResult')  # iframe的id
    page_text = driver.page_source  # 获取页面内容
    tree1 = etree.HTML(page_text)  # etree对象
    sleep(1.5)
    fifty_url = tree1.xpath('//*[@id="id_grid_display_num"]/a[3]/@href')[0]  # 我们设置每页显示50个，有利于快速获取
    new_url = 'https://kns.cnki.net' + fifty_url  # 在selenium中，访问50页会跳往新的页面（因为这里我们是get一个新的url,正常访问知网是不会跳页面的）
    driver.get(new_url)  # 跳转
    sleep(1.5)
    count=0
    for i in range(7):
        contents = driver.find_elements_by_xpath('//a[@class="briefDl_D"]')
        for content in contents:
            count += 1
            content.click()
            #sleep(1)
            windows = driver.window_handles
            driver.switch_to.window(windows[1])
            #sleep(1)
            down = driver.find_element_by_id('Button2')
            down.click()
            sleep(1)
            image = driver.find_element_by_id('VImg')
            image = image.convert('L')
            threshold = 180
            table = []
            for i in range(256):
                if i < threshold:
                    table.append(0)
                else:
                    table.append(1)

            image = image.point(table, '1')
            #image = image.convert('1')
            #image.show()

            result = tesserocr.image_to_text(image)
            vcode_input=driver.find_element_by_id('vcode')
            vcode_input.send_keys(vcode)
            print(result)
            driver.close()
            driver.switch_to.window(windows[0])
            #sleep(1000)
        try:
            next = driver.find_element_by_xpath(r"//div[@class='TitleLeftCell']/a[last()]")
            next.click()
        except:
            None

def get_caj(driver):
    down=driver.find_element_by_id('cajDown')
    down.click()


if __name__ == '__main__':
    auto()

ModuleNotFoundError: No module named 'tesserocr'

In [1]:
import time
import os
from selenium import webdriver
def download(key, page):
    url = "https://www.cnki.net/old" #我们学校的vpn登录网址

    opt = webdriver.ChromeOptions()  # 创建浏览器
    # opt.set_headless()                            #无窗口模式
    # profile.default_content_setting.popups':0 设置为0表示禁止弹出下载窗口
    # 'download.default_directory':"E:\\dir" 修改下载地址为path
    prefs = {'profile.default_content_setting.popups': 0,
             'download.default_directory': path}
    opt.add_experimental_option('prefs', prefs)

    driver = webdriver.Chrome(options=opt)  # 创建浏览器对象
    driver.get(url)  # 打开网页
    # driver.maximize_window()                      #最大化窗口


    # 定位到知网位置并点击

    # 获取所有窗口
    windows = driver.window_handles
    # 切换到当前最新打开的窗口
    driver.switch_to.window(windows[-1])
    # txt_SearchText 为搜索框的位置  键入搜索内容
    driver.find_element_by_id('txt_SearchText').send_keys(key)
    # 定位到搜索按钮的位置并点击
    search = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[1]/input[2]')
    driver.execute_script("arguments[0].click();", search)
    time.sleep(2) #等待加载
    print("ok1")
    try:
        # 点击下载
        # 第一页
        for i in range(1,21):
            # 获取所有窗口
            windows = driver.window_handles
            # 切换到下载页面对应的窗口
            driver.switch_to.window(windows[1])
            try:
                download = '//*[@id="gridTable"]/table/tbody/tr['+str(i)+']/td[9]/a[1]/i'
                download_click = driver.find_element_by_xpath(download)
                driver.execute_script("arguments[0].click();", download_click)
                time.sleep(1)
            except Exception as e:
                continue
        time.sleep(1)
        # 第二到九页
        for k in range(2,page):
            page = 'page' + str(k)
            driver.find_element_by_id(page).click()
            for i in range(1, 21):
                # 获取所有窗口
                windows = driver.window_handles
                # 切换到下载页面对应的窗口
                driver.switch_to.window(windows[1])
                try:
                    download = '//*[@id="gridTable"]/table/tbody/tr[' + str(i) + ']/td[9]/a[1]/i'
                    download_click = driver.find_element_by_xpath(download)
                    driver.execute_script("arguments[0].click();", download_click)
                    time.sleep(1)
                except Exception as e:
                    pass
            time.sleep(1)
    except Exception as e:
        print(e)
    time.sleep(5)
    driver.close()
if __name__ == '__main__':
    kw = input("请输入要搜索的关键词：")
    pg = int(input("请输入要下载的页数："))
    path = r'O:\土壤修复'+'\\'+kw
    if not os.path.exists(path):
        os.makedirs(path)
    try:
        download(kw, pg)
    except Exception as e:
        print(e)


请输入要搜索的关键词：土壤修复
请输入要下载的页数：10
ok1
list index out of range


In [2]:
import time
import xlrd
import os
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from numpy import *

class cnki():
    def __init__(self, save_Path):
        self.save_Path = 'O:\\土壤修复\\{}'.format(save_Path)    # 创建保存文件夹
        # 定义一个无界面的浏览器
        self.options = webdriver.ChromeOptions()
        self.prefs = {'profile.default_content_settings.popups': 0,
                 'profile.managed_default_content_settings.images': 2,
                 'download.default_directory': self.save_Path}
        self.options.add_experimental_option('prefs', self.prefs)
        self.options.add_argument('--headless')
        self.options.add_argument('--disable-gpu')
        self.browser = webdriver.Chrome(executable_path='D:\Chrome\chromedriver.exe', chrome_options=self.options)  
        # 300s无响应就down掉
        self.wait = WebDriverWait(self.browser, 300)
        # 定义窗口最大化
        self.browser.maximize_window()

    def getHtml(self, zy_mc, zy_str):
        try:
            self.browser.get('https://kns.cnki.net/kns8/AdvSearch?dbprefix=SCDB&&crossDbcodes=CJFQ%2CCDMD%2CCIPD%2CCCND%2CCISD%2CSNAD%2CBDZK%2CCJFN%2CCCJD')
            # PDF那个按钮
            self.wait.until(EC.element_to_be_clickable(
                (By.XPATH, '//ul[@class="search-classify-menu"]/li[4]'))).click()
            input = self.wait.until(
                EC.presence_of_element_located((By.XPATH, '//textarea[@class="textarea-major ac_input"]'))
            )
            # 清除里面的数字
            input.clear()
            input.send_keys(zy_str)
            self.wait.until(
                EC.element_to_be_clickable((By.XPATH, '//input[@class="btn-search"]'))
            ).click()
            time.sleep(3)
            total = self.browser.find_element_by_xpath('//*[@id="countPageDiv"]/span/em').text
            print(zy_mc+"一共有"+total+"条数据")
            page = int(total)//20+1
            a = 1
            for p in range(page):
                for i in range(1, 21):
                    link = self.browser.find_element_by_xpath('//*[@id="gridTable"]/table/tbody/tr[%d]/td[2]/a' % i)
                    print(link)
                    flag1 = self.isElementExist('//*[@id="gridTable"]/table/tbody/tr[%d]/td[2]' % i)
                    if flag1:
                        self.browser.execute_script("arguments[0].scrollIntoView();", link)
                        time.sleep(3)
                        actions = ActionChains(self.browser)
                        actions.move_to_element(link)
                        actions.click(link)
                        actions.perform()
                        time.sleep(10)
                        windows = self.browser.window_handles
                        self.browser.switch_to.window(windows[-1])
                        time.sleep(3)
                        try:
                            flag2 = self.isElementExist('//*[@id="pdfDown"]')
                            if flag2:
                                pdf = self.browser.find_element_by_xpath('//*[@id="pdfDown"]')
                                self.browser.execute_script("arguments[0].scrollIntoView();", pdf)
                                time.sleep(3)
                                self.wait.until(EC.presence_of_element_located(
                                    (By.XPATH, '//*[@id="pdfDown"]'))).click()
                            else:
                                pass
                        except Exception as e:
                            print(e)
                        time.sleep(10)
                        self.browser.close()
                        time.sleep(5)
                        self.browser.switch_to.window(windows[0])
                        print("-----正在爬取--" + zy_mc + '--药品的第' + str(int(p)+1) + '页' + str(a) + "条数据------")
                        a = a + 1
                    else:
                        break
                flag3 = self.isElementExist('//*[@id="PageNext"]')
                # 点击下一页
                if flag3:
                    time.sleep(10)
                    next_page = self.browser.find_element_by_xpath('//*[@id="PageNext"]')
                    self.browser.execute_script("arguments[0].scrollIntoView();", next_page)
                    self.wait.until(EC.element_to_be_clickable(
                        (By.XPATH, '//*[@id="PageNext"]'))).click()
                    time.sleep(15)
                else:
                    break
        except Exception as e:
            print(e)
    # 该方法用来确认元素是否存在，如果存在返回flag=true，否则返回false
    def isElementExist(self, element):
        flag = True
        try:
            self.browser.find_element_by_xpath(element)
            return flag
        except:
            flag = False
            return flag

def main():
    try:
        data = xlrd.open_workbook(r'O:\土壤修复\药名.xlsx')
        table = data.sheets()[0]  # 获取第一页
        zymc_lists = table.col_values(0)
        zcy_lists = table.col_values(1)
        for i in range(0, 17):
            str1 = zymc_lists[i]
            str2 = zcy_lists[i]
            print(str2)
            print(30 * '==' + str(i) + 30 * '==')
            c = cnki(str1)
            time.sleep(15)
            c.getHtml(str1, str2)
            time.sleep(5)

    except Exception as e:
        print(e)
    finally:
        pass


if __name__ == '__main__':
    main()


list index out of range
